## Extracción y transformación de datos con Spark 2.4
### Origen de datos
Los datos representan los accidentes de tráfico por día y distrito
que se han producido en la ciudad de Madrid entre 2010 y 2018

El fichero origen es un fichero csv por año tiene los datos detallados
por cada uno de los intervinientes en el accidente: Conductor, pasajero,
testito, peatón, ...

La primera parte del fichero está referida al accidente y se repite
por cada uno de los intervinientes.

### La parte del registro referida al accidente tiene los campos:
 - 'FECHA'
 - 'RANGO HORARIO'
 - 'DIA SEMANA'
 - 'DISTRITO'
 - 'LUGAR ACCIDENTE'
 - 'Nº'
 - 'Nº PARTE'     **Id.accidente, es el mismo para varios regs.**
 - 'CPFA Granizo'
 - 'CPFA Hielo'
 - 'CPFA Lluvia'
 - 'CPFA Niebla'
 - 'CPFA Seco'
 - 'CPFA Nieve'
 - 'CPSV Mojada'
 - 'CPSV Aceite'
 - 'CPSV Barro'
 - 'CPSV Grava Suelta'
 - 'CPSV Hielo'
 - 'CPSV Seca Y Limpia'
 - '* Nº VICTIMAS' **Total de víctimas por Nº de parte**
 - 'TIPO ACCIDENTE'
 - 'Tipo Vehiculo'
 
### La parte del registro correspondiente a cada persona:
 - 'TIPO PERSONA'
 - 'SEXO'
 - 'LESIVIDAD'
 - 'Tramo Edad'


In [1]:
import pyspark
from pyspark.sql import SparkSession
app_name = "accidentes"
master = "local[*]"
spark = (SparkSession.builder
    .master(master)
    .config("spark.driver.cores", 1)
    .appName(app_name)
    .getOrCreate() )
sc = spark.sparkContext
print ('SparkContext created')

SparkContext created


In [2]:
# Cargamos el fichero desde hdfs
#accidenteData = spark.read.csv ('hdfs://localhost:9000/user/ubuntu/accidentes/datos',header=True)
#accidenteData = spark.read.csv ('file:///home/ubuntu/Downloads/*_Accidentalidad.csv-utf8.csv',header=True)
accidenteData = spark.read.csv ('file:///home/jose/Descargas/2018.csv',header=True)

In [3]:
import pyspark.sql.functions as func

# Reducimos las columnas que tenemos que utilizar mediante la Select
# y aplicamos las funciones necesarias a los datos
accRed=accidenteData\
  .filter ((func.substring(accidenteData.LESIVIDAD,1,1)=='H') | 
            (func.substring(accidenteData.LESIVIDAD,1,1)=='M') | 
            (accidenteData['TIPO PERSONA'] == 'CONDUCTOR'))\
  .select(
#        func.to_date(accidenteData.FECHA, 'dd/MM/yyyy HH:mm:ss').alias('fecha'), \
        func.to_date(accidenteData.FECHA, 'dd/MM/yyyy').alias('fecha'), \
#        accidenteData.FECHA.alias('fecha'),\
        accidenteData.DISTRITO.alias('distrito'), \
        accidenteData["Nº PARTE"].alias('idAccidente'), \
        accidenteData["TIPO PERSONA"].alias('tipoPersona'), \
        accidenteData["Tipo Vehiculo"].alias('tipoVehiculo'), \
        accidenteData["SEXO"].alias('sexo'), \
        func.substring(accidenteData.LESIVIDAD,1,2).alias('lesividad'), \
)

In [4]:
#Las columnas del Dataframe accRed quedan así
accRed.printSchema()
#Vemos si alguna columna contiene nulos
accRed.select([func.count(func.when(func.col(c).isNull(), c)).alias(c) for c in accRed.columns]).show()

accRed.show(n=2)


root
 |-- fecha: date (nullable = true)
 |-- distrito: string (nullable = true)
 |-- idAccidente: string (nullable = true)
 |-- tipoPersona: string (nullable = true)
 |-- tipoVehiculo: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- lesividad: string (nullable = true)

+-----+--------+-----------+-----------+------------+----+---------+
|fecha|distrito|idAccidente|tipoPersona|tipoVehiculo|sexo|lesividad|
+-----+--------+-----------+-----------+------------+----+---------+
|    0|       0|          0|          0|           0|   0|        0|
+-----+--------+-----------+-----------+------------+----+---------+

+----------+--------------------+-----------+--------------------+--------------------+------+---------+
|     fecha|            distrito|idAccidente|         tipoPersona|        tipoVehiculo|  sexo|lesividad|
+----------+--------------------+-----------+--------------------+--------------------+------+---------+
|2018-01-01|USERA            ...|     2018/1|PEATON

In [5]:
# Agrupamos por fecha, distrito, accidente y generamos varias columnas contadoras 
accGrouped=accRed\
                .groupBy('fecha','distrito','idAccidente','tipoVehiculo')\
                .agg(func.count(func.when(accRed.lesividad=='MT',1)).alias('muertos'),
                     func.count(func.when(accRed.lesividad=='HG',1)).alias('graves'),
                     func.count(func.when(accRed.lesividad=='HL',1)).alias('leves'),
                     func.count(func.when((accRed.sexo == 'MUJER') &
                                          (accRed.tipoPersona == 'CONDUCTOR'),1)).alias('condMujer'),
                     func.count(func.when((accRed.sexo == 'HOMBRE') &
                                          (accRed.tipoPersona == 'CONDUCTOR'),1)).alias('condHombre'),
                    )\
                .na.fill(0)\
                .sort ('fecha','distrito','idAccidente')

In [6]:
# Añadimos una columna con la indicación "Hombre" cuando los conductores son solo hombres o
# "Mujer" si son solo mujeres, en el resto de casos irá la etiqueta "Ambos"
accGrCondFila=accGrouped.select ("fecha", "distrito", "idAccidente","tipoVehiculo","muertos","graves","leves", "condMujer", "condHombre")\
                .withColumn("conductorHM",
                            func.when (accGrouped.condMujer == 0, "Hombre")
                           .when (accGrouped.condHombre == 0, "Mujer")
                           .otherwise("Ambos")
                )
accGrCondFila.limit(15).toPandas()

,fecha,distrito,idAccidente,tipoVehiculo,muertos,graves,leves,condMujer,condHombre,conductorHM
0,2018-01-01,ARGANZUELA,2018/25,BICICLETA,0,0,1,0,1,Hombre
1,2018-01-01,ARGANZUELA,2018/25,AUTO-TAXI,0,0,0,0,1,Hombre
2,2018-01-01,CARABANCHEL,2018/23,MOTOCICLETA,0,1,0,0,1,Hombre
3,2018-01-01,CARABANCHEL,2018/23,TURISMO,0,0,0,0,1,Hombre
4,2018-01-01,CENTRO,2018/76,BICICLETA,0,0,1,0,1,Hombre
5,2018-01-01,CENTRO,2018/76,NO ASIGNADO,0,0,1,0,0,Hombre
6,2018-01-01,CHAMARTIN,2018/43,TURISMO,0,0,1,0,2,Hombre
7,2018-01-01,CIUDAD LINEAL,2018/12,TURISMO,0,1,0,0,1,Hombre
8,2018-01-01,CIUDAD LINEAL,2018/12,AUTO-TAXI,0,0,2,0,1,Hombre
9,2018-01-01,FUENCARRAL-EL PARDO,2018/16,NO ASIGNADO,0,0,2,0,0,Hombre


#### Por comodidad para poder guardar todos los datos en un único fichero utilizamos la librería pandas.


In [7]:
df2=accGrCondFila.toPandas()
df2.to_csv('~/agrupadoPorAccidenteTipoVehiculo_SexoConductor.csv',sep=';')
